> Abstract: En este notebook regreso a mi crisis de encontrar el mejor modelo para pronosticar la tendencia de una serie de tiempo cuando se pueden hacer tambien listas.

In [39]:
import pandas as pd
import numpy as np

In [40]:
from style import plotly_apply
plotly_apply()

In [41]:
def construccion_de_A(T: int) -> np.ndarray:
    """Construye la matriz A, que es una matriz de diferencias finitas de segundo orden."""
    A = np.zeros((T-2, T))      # Crea una matriz de ceros de tamaño (T-2) x T
    for i in range(T-2):        # Llena la matriz A con los coeficientes de diferencias finitas
        A[i, i:i+3] = 1, -2, 1
    return A    

# Hacerlo una funcion
def hodrick_prescott_filter(series: pd.Series, lamb: float) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Aplica el filtro de Hodrick-Prescott a una serie temporal.
    
    Args:
        series (pd.Series): Serie temporal a la que se le aplicará el filtro.
        lamb (float): Parámetro de suavizamiento del filtro.
        
    Returns:
        tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: Una tupla que contiene la tendencia, el ciclo y la desviación porcentual.
    """
    index = series.index
    name = series.name
    series = np.asarray(series).flatten()

    N = len(series)
    A = construccion_de_A(N)

    tendencia = np.linalg.inv(np.identity(N) + lamb * (A.T @ A)) @ series
    ciclo = series - tendencia
    desviacion = 100 * (series / tendencia - 1)

    tendencia = pd.DataFrame(tendencia, index=index, columns=[name])
    ciclo = pd.DataFrame(ciclo, index=index, columns=[name])
    desviacion = pd.DataFrame(desviacion, index=index, columns=[name])

    return tendencia, ciclo, desviacion

In [137]:
gpd = pd.read_csv('GDP EEUU', index_col=0, parse_dates=True, names=['Fecha', 'GDP'], header=0)
tendencia, ciclo, desviacion = hodrick_prescott_filter(gpd['GDP'], lamb=1600)

In [104]:
gpd['GDP'].plot(title='Producto Interno Bruto Real de Estados Unidos (Billones de Dólares)')
tendencia.plot(title='Tendencia del Producto Interno Bruto Real de Estados Unidos (Billones de Dólares)')
ciclo.plot(title='Ciclo del Producto Interno Bruto Real de Estados Unidos (Billones de Dólares)')
#desviacion.plot(title='Desviación Porcentual del Producto Interno Bruto Real de Estados Unidos (%)')

**Poner la serie en escala logaritmica sirve de tres cosas clave:**

1. Modelo multiplicativo → aditivo.
   Si $Y_t=T_t\cdot C_t$ con ciclo proporcional al nivel, entonces
   $\log Y_t=\tau_t+c_t$. El HP trabaja mejor con $c_t$ de amplitud aproximadamente constante. Sin log, $Y_t=\tau_t+c_t$ impone ciclos de tamaño absoluto fijo, lo que suele ser falso en macro.

2. Estabilizar varianza y hacer ARMA plausible.
   $\operatorname{Var}(Y_t)$ suele crecer con el nivel. En log, $\operatorname{Var}(\log Y_t)$ es más estable. Los residuos del ciclo se acercan a homocedasticidad y a gaussianidad, lo que mejora ajuste y pronóstico ARMA del ciclo.

3. Interpretación económica directa.
   $\Delta \log Y_t \approx g_t$ es tasa de crecimiento. Pronosticar en log produce bandas y errores en porcentajes, que es lo que importa en práctica.

Detalles útiles:

* Requiere $Y_t>0$. Si hay ceros, use $\log(1+Y_t)$ o un shift positivo y documente la interpretación.
* Al volver a niveles, corrija el sesgo por Jensen si asume normalidad: si $\hat y\sim \mathcal N(\mu,\sigma^2)$, entonces $E[Y]\approx \exp(\mu+\tfrac12\sigma^2)$.
* El $\lambda$ del HP no cambia por escalar la serie, pero el log no es un simple reescalado: cambia la relación nivel–volatilidad y hace más lineal la tendencia.


In [60]:
y = np.log(gpd['GDP'])
tendencia, ciclo, desviacion = hodrick_prescott_filter(y, lamb=1600)

**Se hace ARMA en el ciclo porque ARMA exige estacionariedad y el ciclo sí la cumple; la tendencia HP no.**

* Descomposición: $y_t=\tau_t+c_t$, con $E[c_t]=0$, $c_t$ ≈ estacionario; $\tau_t$ es no estacionaria de baja frecuencia.
* Supuesto ARMA: procesos covarianza-estacionarios alrededor de media constante. Eso describe $c_t$, no $\tau_t$.
* El HP impone suavidad vía $\min_{\tau}\sum(y_t-\tau_t)^2+\lambda\sum(\Delta^2\tau_t)^2$. Esa dinámica equivale a un “trend” tipo I(2) local, no a ARMA.
* Poner ARMA sobre $\tau_t$ fuerza a diferenciarla. Modelar $\Delta\tau_t$ o $\Delta^2\tau_t$ destruye la separación tendencia–ciclo y te devuelve a un ARIMA sobre $y_t$.
* Identificación de frecuencias: ARMA sobre $\tau_t$ reintroduce componentes de media cero y “contamina” el ciclo.
* Borde temporal: la $\tau_t$ de HP es inestable en los extremos; ARMA no corrige eso. Un modelo estructural sí.
* Práctica estándar:

  * Ciclo: $c_t=\Phi(B)^{-1}\Theta(B)\varepsilon_t$ (ARMA).
  * Tendencia: local linear trend en espacio de estados
    $\tau_t=\tau_{t-1}+\beta_{t-1}+\eta_t,\quad \beta_t=\beta_{t-1}+\zeta_t$
    o RW con deriva.
* Pronóstico coherente: $\hat y_{T+h}=\hat\tau_{T+h|T}+\hat c_{T+h|T}$.
* Si insistes en “ARMA de tendencia”: ajusta ARMA a $\Delta^2\tau_t$ y luego integra dos veces. Es redundante frente a ARIMA/estructurales y pierde claridad.


In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
ciclo = ciclo.asfreq('QS-JAN')  # evita errores

In [150]:
def get_all_combinations(p: int) -> list[tuple[int, ...]]:
    """Genera todas las combinaciones posibles de ceros y unos de longitud hasta p, 
    asegurando que cada combinación termine en 1, y que la combinación (0,) esté incluida.
    
    Args:
        p (int): La longitud máxima de las combinaciones.
        
    Returns:
        list[tuple[int, ...]]: Una lista de tuplas que representan las combinaciones.
    """
    all_combinations = []
    all_combinations.append((0,))  # Include the (0,) combination
    for i in range(1, p+1):
        for comb in product([0, 1], repeat=i):
            if comb[-1] == 1:  # Only include combinations ending with 1
                all_combinations.append(comb)
    return all_combinations

In [152]:
p_combs = get_all_combinations(3)
p_combs

[(0,), (1,), (0, 1), (1, 1), (0, 0, 1), (0, 1, 1), (1, 0, 1), (1, 1, 1)]

In [ ]:
ARIMA(ciclo['GDP'], order=(1, 0, [0, 0, 1]), ).fit().summary() # solo usa el AR 3

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                    GDP   No. Observations:                  314
Model:               ARIMA(1, 0, [3])   Log Likelihood               -1974.829
Date:                Mon, 08 Sep 2025   AIC                           3957.657
Time:                        02:16:02   BIC                           3972.655
Sample:                    01-01-1947   HQIC                          3963.650
                         - 04-01-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0068     35.025      0.000      1.000     -68.641      68.655
ar.L1          0.6196      0.018     34.865      0.000       0.585       0.654
ma.L3          0.0188      0.088      0.214      0.830      -0.153       0.191
sigma2      1.697e+04    408.053     41.579      0.000    1.62e+04    1.78e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            158235.04
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):              27.22   Skew:                            -7.59
Prob(H) (two-sided):                  0.00   Kurtosis:                       111.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [153]:
best = (np.inf, None, None)

for p in p_combs:
    for q in range(0, 4):
        try:
            m = ARIMA(ciclo['GDP'], order=(p, 0, q)).fit()
            if m.aic < best[0]:
                best = (m.aic, (p, q), m)
        except Exception:
            pass

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\

In [155]:
aic, (p, q), arma = best
print(f'El mejor modelo es ARMA({p},{q}) con AIC={aic:.2f}')
arma.summary()

El mejor modelo es ARMA((0, 1, 1),2) con AIC=12.00


c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:3016: RuntimeWarning:

divide by zero encountered in divide

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\stattools.py:1431: RuntimeWarning:

invalid value encountered in divide

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\stattools.py:702: RuntimeWarning:

invalid value encountered in divide



<class 'statsmodels.iolib.summary.Summary'>
"""
                                SARIMAX Results                                
===============================================================================
Dep. Variable:                     GDP   No. Observations:                  314
Model:             ARIMA([2, 3], 0, 2)   Log Likelihood                   0.000
Date:                 Mon, 08 Sep 2025   AIC                             12.000
Time:                         02:24:17   BIC                             34.496
Sample:                     01-01-1947   HQIC                            20.989
                          - 04-01-2025                                         
Covariance Type:                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.2963         -0       -inf      0.000      19.296      19.296
ar.L2          1.0097         -0       -inf      0.000       1.010       1.010
ar.L3         -0.0230         -0        inf      0.000      -0.023      -0.023
ma.L1         -0.5300         -0        inf      0.000      -0.530      -0.530
ma.L2         -0.4699         -0        inf      0.000      -0.470      -0.470
sigma2      1.414e+04         -0       -inf      0.000    1.41e+04    1.41e+04
===================================================================================
Ljung-Box (L1) (Q):                    nan   Jarque-Bera (JB):                  nan
Prob(Q):                               nan   Prob(JB):                          nan
Heteroskedasticity (H):                nan   Skew:                              nan
Prob(H) (two-sided):                   nan   Kurtosis:                          nan
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number    inf. Standard errors may be unstable.
"""

In [157]:
ARIMA(ciclo['GDP'], order=(3, 0, 3)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                    GDP   No. Observations:                  314
Model:                 ARIMA(3, 0, 3)   Log Likelihood               -1956.902
Date:                Mon, 08 Sep 2025   AIC                           3929.805
Time:                        02:25:28   BIC                           3959.800
Sample:                    01-01-1947   HQIC                          3941.790
                         - 04-01-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0722      1.354     -0.053      0.958      -2.727       2.582
ar.L1          0.7309      0.112      6.523      0.000       0.511       0.950
ar.L2          0.8885      0.111      8.011      0.000       0.671       1.106
ar.L3         -0.7527      0.090     -8.345      0.000      -0.930      -0.576
ma.L1         -0.2300      2.158     -0.107      0.915      -4.460       4.001
ma.L2         -0.9997      0.530     -1.887      0.059      -2.038       0.039
ma.L3          0.2297      0.093      2.476      0.013       0.048       0.411
sigma2      1.487e+04      0.000   1.26e+08      0.000    1.49e+04    1.49e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):            158160.79
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):              26.51   Skew:                            -7.49
Prob(H) (two-sided):                  0.00   Kurtosis:                       111.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 4.67e+24. Standard errors may be unstable.
"""

In [159]:
ARIMA(ciclo['GDP'], order=([1, 1, 1], 0, [1, 1, 1])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                    GDP   No. Observations:                  314
Model:                 ARIMA(3, 0, 3)   Log Likelihood               -1956.902
Date:                Mon, 08 Sep 2025   AIC                           3929.805
Time:                        02:25:48   BIC                           3959.800
Sample:                    01-01-1947   HQIC                          3941.790
                         - 04-01-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0722      1.354     -0.053      0.958      -2.727       2.582
ar.L1          0.7309      0.112      6.523      0.000       0.511       0.950
ar.L2          0.8885      0.111      8.011      0.000       0.671       1.106
ar.L3         -0.7527      0.090     -8.345      0.000      -0.930      -0.576
ma.L1         -0.2300      2.158     -0.107      0.915      -4.460       4.001
ma.L2         -0.9997      0.530     -1.887      0.059      -2.038       0.039
ma.L3          0.2297      0.093      2.476      0.013       0.048       0.411
sigma2      1.487e+04      0.000   1.26e+08      0.000    1.49e+04    1.49e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):            158160.79
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):              26.51   Skew:                            -7.49
Prob(H) (two-sided):                  0.00   Kurtosis:                       111.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 4.67e+24. Standard errors may be unstable.
"""

In [165]:
ARIMA(ciclo['GDP'], order=([0, 0, 1], 0, [0, 0, 1])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                    GDP   No. Observations:                  314
Model:             ARIMA([3], 0, [3])   Log Likelihood               -2042.175
Date:                Mon, 08 Sep 2025   AIC                           4092.349
Time:                        02:27:47   BIC                           4107.347
Sample:                    01-01-1947   HQIC                          4098.342
                         - 04-01-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0033     15.572     -0.000      1.000     -30.523      30.516
ar.L3         -0.1588      0.255     -0.624      0.533      -0.658       0.340
ma.L3          0.4148      0.242      1.711      0.087      -0.060       0.890
sigma2      2.608e+04    629.098     41.455      0.000    2.48e+04    2.73e+04
===================================================================================
Ljung-Box (L1) (Q):                 101.75   Jarque-Bera (JB):             42616.53
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              16.84   Skew:                            -5.11
Prob(H) (two-sided):                  0.00   Kurtosis:                        59.15
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [89]:


# ARMA(p,q) para el ciclo con selección por AIC en p,q <= 3
best = (np.inf, None, None)     # Crear una tupla para guardar el mejor modelo (AIC, (p,q), modelo)
for p in range(0, 4):           # Probar todos los pares (p,q) con p,q en {0,1,2,3}
    for q in range(0, 4):
        if p == 0 and q == 0:   # Evitar el modelo ARMA(0,0)
            continue
        try:
            m = ARIMA(ciclo['GDP'], order=(p, 0, q)).fit()  
            if m.aic < best[0]: # Si el AIC es mejor (menor) que el mejor hasta ahora, actualizar la tupla
                best = (m.aic, (p, q), m)
        except Exception:
            pass

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmode

In [96]:
aic, (p, q), arma = best
print(f'El mejor modelo es ARMA({p},{q}) con AIC={aic:.2f}')
arma.summary()

El mejor modelo es ARMA(2,1) con AIC=-2035.76


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                    GDP   No. Observations:                  314
Model:                 ARIMA(2, 0, 1)   Log Likelihood                1022.881
Date:                Mon, 08 Sep 2025   AIC                          -2035.763
Time:                        01:43:50   BIC                          -2017.016
Sample:                    01-01-1947   HQIC                         -2028.272
                         - 04-01-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.136e-05   9.66e-05      0.118      0.906      -0.000       0.000
ar.L1          1.7741      0.028     63.802      0.000       1.720       1.829
ar.L2         -0.8559      0.025    -33.764      0.000      -0.906      -0.806
ma.L1         -1.0000     14.268     -0.070      0.944     -28.966      26.966
sigma2      8.513e-05      0.001      0.070      0.944      -0.002       0.002
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):              6641.30
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               1.33   Skew:                            -1.41
Prob(H) (two-sided):                  0.14   Kurtosis:                        25.35
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [99]:
ljung_box = sm.stats.acorr_ljungbox(arma.resid, lags=10, return_df=True)
ljung_box

,lb_stat,lb_pvalue
1,0.009760,0.921302
2,0.538569,0.763926
3,0.636876,0.887941
4,0.978653,0.913015
5,1.043963,0.958943
6,1.043981,0.983890
7,1.181554,0.991345
8,1.614375,0.990642
9,3.238935,0.954064
10,4.861435,0.900240


In [ ]:
# plot tho autocorrelation of residuals
sm.graphics.tsa.plot_acf(arma.resid, lags=20, alpha=
                         

In [ ]:

aic, (p, q), arma = best
fc_c = arma.get_forecast(h)
c_mean = fc_c.predicted_mean
c_ci = fc_c.conf_int(alpha=0.2)  # banda 80

**El ciclo HP debe tener media cero y revertir a cero, se entrena con n**

Porque el **ciclo HP** es, por construcción, una serie **de media cero y sin deriva determinista**.
En el HP: $y_t=\tau_t+c_t$ y la penalización $\sum(\Delta^2\tau_t)^2$ tiene núcleo $\{\text{constante}+ \text{tendencia lineal}\}$. Las CPO implican que $c_t$ es **ortogonal** a ese núcleo, i.e. $\sum c_t\approx0$ y $\sum t\,c_t\approx0$. Por eso el componente cíclico debe **revertir a 0**.

Si incluyes $\text{trend='c' o 't'}$ en el ARMA, el modelo “roba” parte de la tendencia y rompe la separación tendencia–ciclo, además de sesgar el pronóstico del ciclo lejos de cero.

Evidencia empírica en tu salida: la constante no es significativa. Coherente con $\text{trend='n'}$.

Qué hacer si tu ciclo no parece centrado:

* Demean: $c_t \leftarrow c_t - \bar c$.
* Revisa $\lambda$ y bordes; el sesgo de extremos puede desplazar la media.
* Sólo si $\bar c$ es estadísticamente diferente de 0, usa `trend='c'` como parche, pero documenta que ya no es un “ciclo HP puro”.
Significado de `trend` en `statsmodels` (ARIMA/SARIMAX):

- `trend='n'` → **sin término determinista**. Media cero en $d=0$. En un ciclo HP debe usarse esto.  
- `trend='c'` → **constante** $\mu$. En ARMA ($d=0$) es una media no nula. En ARIMA ($d\ge1$) la constante induce **deriva**: una tendencia polinómica de grado $d$ en niveles.  
- `trend='t'` → **tendencia lineal sin intercepto**: coeficiente $\delta\cdot t$.  
- `trend='ct'` → **intercepto + tendencia lineal**: $\mu+\delta t$.  
- “**polinomial**” → en ARIMA/SARIMAX no se pasa como cadena; se construye con **regresores exógenos** $1,t,t^2,\dots$ (o con `DeterministicProcess`). En OLS existe además `ctt` (constante, lineal y cuadrática), pero ARIMA/SARIMAX estándar acepta solo `n`,`c`,`t`,`ct`.

**Efectos formales**

Con ARIMA $(p,d,q)$ :

$$
\Phi(L)(1-L)^d y_t=\mu\cdot \mathbf 1\{trend\ \text{incluye }c\}+\Theta(L)\varepsilon_t.
$$

- Si $d=0$ y `c`: $E[y_t]=\dfrac{\mu}{1-\sum_{i=1}^p\phi_i}$.  
- Si $d=1$ y `c`: $y_t$ tiene **pendiente** $\kappa=\dfrac{\mu}{1-\sum\phi_i}$ (drift).  
- Si $d=2$ y `c`: $y_t$ tiene tendencia **cuadrática**; en general, el grado del polinomio determinista es $d$.

Con `t` o `ct`, se añaden términos $\delta t$ (y eventualmente $\mu$) al lado derecho. En $d>0$ esto se acumula en niveles como polinomios de grado $d+1$ o $d$.

### Cuándo usar cada uno
- **Ciclo HP o series ya centradas**: `n`.  
- **ARMA en rendimientos o log-diferencias**: normalmente `n` o, si hay sesgo, `c`.  
- **Niveles con paseo aleatorio ($d=1$)**: `c` para permitir drift. Evita simultanear `ct` con diferenciar $d=1$ si ese crecimiento ya es estocástico.  
- **Tendencias deterministas claras** en niveles estacionarios ($d=0$): `t` o `ct`.  
- **Polinomios de mayor grado**: usa exógenas; no se indican con el string `trend`.

### Polinomio en práctica (ARIMA/SARIMAX)
```python
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.arima.model import ARIMA

idx = y.index  # DatetimeIndex regular
dp = DeterministicProcess(index=idx, constant=True, order=2)  # 1, t, t^2
X = dp.in_sample()
res = ARIMA(y, order=(p,d,q), exog=X).fit()

Xf = dp.out_of_sample(steps=h)   # para pronóstico
fc = res.get_forecast(steps=h, exog=Xf)
```

### Advertencias técnicas
- Un término determinista puede **competir** con la parte integrada: decide si la tendencia es estocástica (diferenciación) o determinista (trend), evitando redundancia.  
- En ciclos, añadir `c` o `t` **contamina** la separación tendencia–ciclo.  
- La inclusión de polinomios altos aumenta colinealidad y varianza de estimación. Usa criterios de información y pruebas de diagnóstico.


Resumen práctico: **más alto** log-likelihood es mejor; **más bajo** AIC/BIC/HQIC es mejor. Tus números implican $T=314$ y $k=5$ parámetros (const, AR1, AR2, MA1, $\sigma^2$); se verifica porque
$-2\ell+2k=-2035.763\Rightarrow k\approx5$.

# Definiciones, fórmula, intuición y uso

## Log Likelihood $\ell(\hat\theta)$

**Qué es.** Verosimilitud del modelo en los datos con parámetros estimados.
**Fórmula (general).**

$$
\ell(\theta)=\sum_{t=1}^{T}\log f_\theta(y_t\mid\mathcal F_{t-1}).
$$

Para ARIMA gaussiano vía filtro de innovaciones:

$$
\ell(\theta)=-\tfrac{T}{2}\log(2\pi)-\tfrac{1}{2}\sum_{t=1}^{T}\Big(\log v_t+\tfrac{\varepsilon_t^2}{v_t}\Big),
$$

donde $\varepsilon_t$ y $v_t$ son error y varianza de predicción 1-paso.
**Intuición.** Cuánto “probable” hace el modelo a los datos.
**Cuándo usar.** Nunca solo para comparar entre modelos con distinto $k$; úsalo como insumo de AIC/BIC/HQIC.

## AIC (Akaike Information Criterion)

**Fórmula.**

$$
\mathrm{AIC}=2k-2\ell(\hat\theta).
$$

**Intuición.** Trade-off ajuste-complejidad con penalización **constante** por parámetro; aproxima el riesgo de predicción fuera de muestra (K-L risk).
**Uso recomendado.** Selección enfocada en **pronóstico** cuando el “modelo verdadero” puede ser complejo o no está en la familia candidata. Preferir **AICc** en muestras pequeñas:

$$
\mathrm{AICc}=\mathrm{AIC}+\frac{2k(k+1)}{T-k-1}.
$$

**Regla práctica.** Comparar $\Delta_i=\mathrm{AIC}_i-\mathrm{AIC}_{\min}$; $\Delta\le2$ soporte sustancial, $4\!-\!7$ moderado, $\ge 10$ casi nulo. Pesos de Akaike:

$$
w_i=\frac{\exp(-\Delta_i/2)}{\sum_j \exp(-\Delta_j/2)}.
$$

## BIC (Bayesian Information Criterion, Schwarz)

**Fórmula.**

$$
\mathrm{BIC}=k\log T-2\ell(\hat\theta).
$$

**Intuición.** Penalización **creciente con $T$**; aproximación de la evidencia bayesiana (margen) con prior “unit information”.
**Uso recomendado.** **Identificación estructural** y parsimonia; es **consistente** si el verdadero modelo finito está entre los candidatos.
**Regla práctica.** Diferencias $\Delta\mathrm{BIC}$ se interpretan como log-Bayes factors aproximados: evidencia positiva $2\!-\!6$, fuerte $6\!-\!10$, muy fuerte $>10$.

## HQIC (Hannan–Quinn)

**Fórmula.**

$$
\mathrm{HQIC}=2k\log\log T-2\ell(\hat\theta).
$$

**Intuición.** Penalización intermedia entre AIC y BIC; **consistente** como BIC pero menos severo en muestras moderadas.
**Uso recomendado.** Cuando BIC parece “sub-ajustar” y AIC “sobre-ajustar”; punto medio prudente.

# Verificación con tus cifras

$$
\begin{aligned}
\ell&=1022.881, \quad T=314,\quad k=5,\\
\mathrm{AIC}&=2(5)-2(1022.881)=-2035.763,\\
\mathrm{BIC}&=5\ln(314)-2(1022.881)=-2017.016,\\
\mathrm{HQIC}&=2(5)\ln\ln(314)-2(1022.881)=-2028.272.
\end{aligned}
$$

# ¿Cuál conviene?

* **Pronóstico puro / modelos candidatos todos “mal-especificados”:** AIC o AICc.
* **Parquedad y posible “modelo verdadero” finito:** BIC.
* **Compromiso ajuste-parsimonia cuando AIC y BIC discrepan:** HQIC.
* **Series muy largas:** BIC tiende a penalizar más, favoreciendo órdenes bajos.
* **Series cortas:** preferir AICc.

Siempre seleccionar por el **criterio mínimo** y luego validar con diagnósticos: residuos blancos, invertibilidad/estacionariedad, ausencia de ARCH, estabilidad de parámetros y backtesting.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from scipy.stats import jarque_bera

def _roots_ok(m, margin=1.02):
    """Verifica que las raíces del modelo ARMA estén fuera del círculo unitario con un margen dado, el margen por defecto es 1.02, significa que las raíces deben estar al menos al 102% de la distancia del círculo unitario, una raíz en el círculo unitario tiene valor absoluto 1."""
    # raíces fuera del círculo unitario con margen
    ok_ar = (len(m.arroots)==0) or (np.min(np.abs(m.arroots)) > margin)
    ok_ma = (len(m.maroots)==0) or (np.min(np.abs(m.maroots)) > margin)
    return ok_ar and ok_ma

def _resid_tests(resid, lags=(12, 24), alpha=0.05):
    resid = pd.Series(resid).dropna()
    # Blanco: Ljung–Box en varios lags
    lb = acorr_ljungbox(resid, lags=max(lags), return_df=True)
    ok_lb = all(lb.loc[lag, 'lb_pvalue'] > alpha for lag in lags if lag <= len(lb))
    # ARCH: no heterocedasticidad condicional
    arch_stat, arch_p, _, _ = het_arch(resid, nlags=min(12, max(5, len(resid)//20)))
    ok_arch = arch_p > alpha
    # Normalidad: Jarque–Bera (criterio suave)
    jb_stat, jb_p, skew, kurt = jarque_bera(resid)
    return {
        'ok_lb': ok_lb, 'ok_arch': ok_arch, 'ok_jb': jb_p > alpha,
        'jb_p': jb_p, 'arch_p': arch_p,
        'lb_p@{}': float(lb.iloc[min(max(lags), len(lb))-1]['lb_pvalue'])
    }

def seleccionar_arma_ciclo(serie, max_p=3, max_q=3, alpha=0.05):
    """
    serie: ciclo HP (media ~0), Index temporal regular.
    Devuelve el mejor modelo que CUMPLE supuestos. Fallback: mejor AIC con motivos.
    """
    y = pd.Series(serie).dropna()
    if y.index.freq is None:
        y = y.asfreq(pd.infer_freq(y.index))

    candidatos = []
    fallback = None

    for p, q in product(range(max_p+1), range(max_q+1)):        # se usa product para generar pares (p,q)
        try:
            # Tendencia nula en ciclo; forzar restricciones
            mod = ARIMA(y, order=(p, 0, q), trend='n',
                        enforce_stationarity=True, enforce_invertibility=True)
            res = mod.fit()
            # guardar fallback por AIC
            if fallback is None or res.aic < fallback[0]:
                fallback = (res.aic, (p, q), res, None)

            # raíces con margen
            if not _roots_ok(res):
                continue

            # diagnósticos de residuos
            tests = _resid_tests(res.resid, lags=(12, 24), alpha=alpha)
            # criterio: blanco y sin ARCH; normalidad opcional pero preferida
            if tests['ok_lb'] and tests['ok_arch']:
                # penaliza si no normal, pero acepta
                score = res.aic + (0 if tests['ok_jb'] else 2.0)
                candidatos.append((score, (p, q), res, tests))
        except Exception:
            continue

    if candidatos:
        candidatos.sort(key=lambda z: z[0])
        return {'order': candidatos[0][1], 'result': candidatos[0][2], 'tests': candidatos[0][3], 'fallback_used': False}
    else:
        return {'order': fallback[1], 'result': fallback[2], 'tests': fallback[3], 'fallback_used': True}


In [102]:
sel = seleccionar_arma_ciclo(y, max_p=3, max_q=3, alpha=0.1)

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

c:\Users\herie\miniconda3\envs\u\Lib\site-

In [103]:
sel

{'order': (1, 2),
 'result': <statsmodels.tsa.arima.model.ARIMAResultsWrapper at 0x2d4b4896bd0>,
 'tests': None,
 'fallback_used': True}

In [108]:
ARIMA(ciclo, order=(1, 0, [1, 0, 1, 1]), trend='n').fit().summary()

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.

c:\Users\herie\miniconda3\envs\u\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency QS-OCT will be used.



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                  
==================================================================================
Dep. Variable:                        GDP   No. Observations:                  314
Model:             ARIMA(1, 0, [1, 3, 4])   Log Likelihood                1002.601
Date:                    Mon, 08 Sep 2025   AIC                          -1995.203
Time:                            01:54:12   BIC                          -1976.456
Sample:                        01-01-1947   HQIC                         -1987.712
                             - 04-01-2025                                         
Covariance Type:                      opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7616      0.074     10.319      0.000       0.617       0.906
ma.L1          0.1232      0.081      1.518      0.129      -0.036       0.282
ma.L3         -0.0588      0.083     -0.706      0.480      -0.222       0.104
ma.L4         -0.1009      0.080     -1.261      0.207      -0.258       0.056
sigma2      9.824e-05   2.66e-06     36.937      0.000     9.3e-05       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.09   Jarque-Bera (JB):              5916.20
Prob(Q):                              0.76   Prob(JB):                         0.00
Heteroskedasticity (H):               1.29   Skew:                            -1.59
Prob(H) (two-sided):                  0.20   Kurtosis:                        24.02
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:

def pronosticar_ciclo(res, h=12, alpha=0.2):
    fc = res.get_forecast(h)
    mean = fc.predicted_mean
    ci = fc.conf_int(alpha=alpha)
    return mean, ci

def plot_ciclo_con_pronostico(ciclo, res, h=12, alpha=0.2, titulo='Ciclo HP + ARMA'):
    mean, ci = pronosticar_ciclo(res, h=h, alpha=alpha)
    idx_hist = ciclo.index
    idx_fc = mean.index

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.axhline(0.0, linewidth=0.8)
    ax.plot(idx_hist, ciclo, label='Ciclo HP')
    ax.plot(idx_fc, mean, label='Pronóstico ciclo', linewidth=1.8)
    ax.fill_between(idx_fc, ci.iloc[:, 0], ci.iloc[:, 1], alpha=0.2, step='pre', label=f'Banda {int((1-alpha)*100)}%')
    ax.set_title(titulo)
    ax.legend()
    plt.tight_layout()
    return fig

# === Uso típico ===
# y_ciclo = ciclo['GDP']  # tu serie del componente cíclico
# sel = seleccionar_arma_ciclo(y_ciclo, max_p=3, max_q=3, alpha=0.05)
# print("Orden seleccionado:", sel['order'], "fallback:", sel['fallback_used'])
# if sel['tests'] is not None:
#     print(sel['tests'])
# fig = plot_ciclo_con_pronostico(y_ciclo, sel['result'], h=12, alpha=0.2, titulo='Ciclo y pronóstico')
# plt.show()
